In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%watermark -p onnx,onnxruntime,tf2onnx
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


numpy 1.19.5
sklearn not installed
pandas 1.1.5
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
torch not installed
torchvision not installed
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
onnx 1.10.1
onnxruntime 1.8.1
tf2onnx 1.9.2


In [2]:
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'tensorboard-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_cell_magic
def template_writefile(line, cell):
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), 600
    if len(args) == 3:
        height = int(args[2])
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    netron.start(file, address=('0.0.0.0', port), browse=False)
    display_html(port, height)


In [23]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [24]:
#@title
import base64
import io
import os
import time

from scipy.signal import medfilt
import cv2


from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript

import matplotlib
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image


# import tensorflow.compat.v2 as tf

# ! pip install youtube_dl
# import youtube_dl

# from google.colab import drive
# from google.colab import output
# from google.colab.output import eval_js

# Model definition
layers = tf.keras.layers
regularizers = tf.keras.regularizers


def get_sims(embs, temperature):
  """Calculates self-similarity between batch of sequence of embeddings."""
  batch_size = tf.shape(embs)[0]
  seq_len = tf.shape(embs)[1]
  embs = tf.reshape(embs, [batch_size, seq_len, -1])

  def _get_sims(embs):
    """Calculates self-similarity between sequence of embeddings."""
    dist = pairwise_l2_distance(embs, embs)
    sims = -1.0 * dist
    return sims

  sims = tf.map_fn(_get_sims, embs)
  sims /= temperature
  sims = tf.nn.softmax(sims, axis=-1)
  sims = tf.expand_dims(sims, -1)
  return sims


def flatten_sequential_feats(x, batch_size, seq_len):
  """Flattens sequential features with known batch size and seq_len."""
  x = tf.reshape(x, [batch_size, seq_len, -1])
  return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.

  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    outputs: shape == (..., seq_len_q, depth_v)
    attention_weights: shape == (..., seq_len_q, seq_len_k)
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk.
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  # (..., seq_len_q, seq_len_k)
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])


class MultiHeadAttention(tf.keras.layers.Layer):
  """Multi-headed attention layer."""

  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth)."""
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(
        q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(
        k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(
        v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(
        scaled_attention,
        perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(
        scaled_attention,
        (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
  """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
  """

  def __init__(self, d_model, num_heads, dff,
               dropout_rate=0.1,
               reorder_ln=False):
    super(TransformerLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.reorder_ln = reorder_ln

  def call(self, x):
    inp_x = x

    if self.reorder_ln:
      x = self.layernorm1(x)

    # (batch_size, input_seq_len, d_model)
    attn_output, _ = self.mha(x, x, x, mask=None)
    attn_output = self.dropout1(attn_output)

    if self.reorder_ln:
      out1 = inp_x + attn_output
      x = out1
    else:
      # (batch_size, input_seq_len, d_model)
      out1 = self.layernorm1(x + attn_output)
      x = out1

    if self.reorder_ln:
      x = self.layernorm2(x)

    # (batch_size, input_seq_len, d_model)
    ffn_output = self.ffn(x)
    ffn_output = self.dropout2(ffn_output)

    if self.reorder_ln:
      out2 = out1 + ffn_output
    else:
      # (batch_size, input_seq_len, d_model)
      out2 = self.layernorm2(out1 + ffn_output)

    return out2


def pairwise_l2_distance(a, b):
  """Computes pairwise distances between all rows of a and all rows of b."""
  norm_a = tf.reduce_sum(tf.square(a), 1)
  norm_a = tf.reshape(norm_a, [-1, 1])
  norm_b = tf.reduce_sum(tf.square(b), 1)
  norm_b = tf.reshape(norm_b, [1, -1])
  dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
  return dist


def get_repnet_model(logdir):
  """Returns a trained RepNet model.

  Args:
    logdir (string): Path to directory where checkpoint will be downloaded.

  Returns:
    model (Keras model): Trained RepNet model.
  """
  # Check if we are in eager mode.
  assert tf.executing_eagerly()

  # Models will be called in eval mode.
  tf.keras.backend.set_learning_phase(0)

  # Define RepNet model.
  model = ResnetPeriodEstimator()
  # tf.function for speed.
  model.call = tf.function(model.call)

  # Define checkpoint and checkpoint manager.
  ckpt = tf.train.Checkpoint(model=model)
  ckpt_manager = tf.train.CheckpointManager(
      ckpt, directory=logdir, max_to_keep=10)
  latest_ckpt = ckpt_manager.latest_checkpoint
  print('Loading from: ', latest_ckpt)
  if not latest_ckpt:
    raise ValueError('Path does not have a checkpoint to load.')
  # Restore weights.
  ckpt.restore(latest_ckpt).expect_partial()

  # Pass dummy frames to build graph.
  model(tf.random.uniform((1, 64, 112, 112, 3)))
  return model


def unnorm(query_frame):
  min_v = query_frame.min()
  max_v = query_frame.max()
  query_frame = (query_frame - min_v) / max(1e-7, (max_v - min_v))
  return query_frame

class ResnetPeriodEstimator(tf.keras.models.Model):
  """RepNet model."""

  def __init__(
      self,
      num_frames=64,
      image_size=112,
      base_model_layer_name='conv4_block3_out',
      temperature=13.544,
      dropout_rate=0.25,
      l2_reg_weight=1e-6,
      temporal_conv_channels=512,
      temporal_conv_kernel_size=3,
      temporal_conv_dilation_rate=3,
      conv_channels=32,
      conv_kernel_size=3,
      transformer_layers_config=((512, 4, 512),),
      transformer_dropout_rate=0.0,
      transformer_reorder_ln=True,
      period_fc_channels=(512, 512),
      within_period_fc_channels=(512, 512)):
    super(ResnetPeriodEstimator, self).__init__()

    # Model params.
    self.num_frames = num_frames
    self.image_size = image_size

    self.base_model_layer_name = base_model_layer_name

    self.temperature = temperature

    self.dropout_rate = dropout_rate
    self.l2_reg_weight = l2_reg_weight

    self.temporal_conv_channels = temporal_conv_channels
    self.temporal_conv_kernel_size = temporal_conv_kernel_size
    self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

    self.conv_channels = conv_channels
    self.conv_kernel_size = conv_kernel_size
    # Transformer config in form of (channels, heads, bottleneck channels).
    self.transformer_layers_config = transformer_layers_config
    self.transformer_dropout_rate = transformer_dropout_rate
    self.transformer_reorder_ln = transformer_reorder_ln

    self.period_fc_channels = period_fc_channels
    self.within_period_fc_channels = within_period_fc_channels

    # Base ResNet50 Model.
    base_model = tf.keras.applications.ResNet50V2(
        include_top=False, weights=None, pooling='max')
    self.base_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(self.base_model_layer_name).output)

    # 3D Conv on k Frames
    self.temporal_conv_layers = [
        layers.Conv3D(self.temporal_conv_channels,
                      self.temporal_conv_kernel_size,
                      padding='same',
                      dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                      kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                      kernel_initializer='he_normal', name='RE_temporal_conv_layers')]
    self.temporal_bn_layers = [layers.BatchNormalization(name='RE_temporal_bn_layers')
                               for _ in self.temporal_conv_layers]

    # Counting Module (Self-sim > Conv > Transformer > Classifier)
    self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                        self.conv_kernel_size,
                                        padding='same',
                                        activation=tf.nn.relu, name='RE_conv_3x3_layer')

    channels = self.transformer_layers_config[0][0]
    self.input_projection = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None, name='RE_input_projection')

    self.input_projection2 = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None, name='RE_input_projection2')

    length = self.num_frames
    self.pos_encoding = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
    self.pos_encoding2 = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding2',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

    self.transformer_layers = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    self.transformer_layers2 = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers2.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    # Period Prediction Module.
    self.dropout_layer = layers.Dropout(self.dropout_rate, name='RE_dropout_layer')
    num_preds = self.num_frames//2
    self.fc_layers = []
    for i, channels in enumerate(self.period_fc_channels):
      self.fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu, name=f'RE_fc_layers_{i}'))
    self.fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight), name='RE_fc_layers_x'))

    # Within Period Module
    num_preds = 1
    self.within_period_fc_layers = []
    for i, channels in enumerate(self.within_period_fc_channels):
      self.within_period_fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu, name=f'RE_within_period_fc_layers_{i}'))
    self.within_period_fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight), name='RE_within_period_fc_layers_x'))

  def call(self, x):
    # Ensures we are always using the right batch_size during train/eval.
    batch_size = tf.shape(x)[0]
    # Conv Feature Extractor.
    x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
    x = self.base_model(x)
    h = tf.shape(x)[1]
    w = tf.shape(x)[2]
    c = tf.shape(x)[3]
    x = tf.reshape(x, [batch_size, -1, h, w, c])

    # 3D Conv to give temporal context to per-frame embeddings. 
    for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                    self.temporal_conv_layers):
      print('1conv_layer:', x.shape)
      x = conv_layer(x)
      print('2conv_layer:', x.shape)
      x = bn_layer(x)
      print('bn_layer:', x.shape)
      x = tf.nn.relu(x)
      print('relu layer:', x.shape)
    

    x = tf.reduce_max(x, [2, 3])
    

    # Reshape and prepare embs for output.
    final_embs = x

    # Get self-similarity matrix.
    x = get_sims(x, self.temperature)
    
    # 3x3 conv layer on self-similarity matrix.
    x = self.conv_3x3_layer(x)
    x = tf.reshape(x, [batch_size, self.num_frames, -1])
    within_period_x = x

    # Period prediction.
    x = self.input_projection(x)
    x += self.pos_encoding
    for transformer_layer in self.transformer_layers:
      x = transformer_layer(x)
    x = flatten_sequential_feats(x, batch_size, self.num_frames)
    for fc_layer in self.fc_layers:
      x = self.dropout_layer(x)
      x = fc_layer(x)

    # Within period prediction.
    within_period_x = self.input_projection2(within_period_x)
    within_period_x += self.pos_encoding2
    for transformer_layer in self.transformer_layers2:
      within_period_x = transformer_layer(within_period_x)
    within_period_x = flatten_sequential_feats(within_period_x,
                                               batch_size,
                                               self.num_frames)
    for fc_layer in self.within_period_fc_layers:
      within_period_x = self.dropout_layer(within_period_x)
      within_period_x = fc_layer(within_period_x)

    return x, within_period_x, final_embs

#   @tf.function
#   def preprocess(self, imgs):
#     imgs = tf.cast(imgs, tf.float32)
#     imgs -= 127.5
#     imgs /= 127.5
#     imgs = tf.image.resize(imgs, (self.image_size, self.image_size))
#     return imgs

In [25]:
model1 = ResnetPeriodEstimator()
print(model1.pos_encoding.shape)
print(model1.pos_encoding[0][0][:10])
ckpt = tf.train.Checkpoint(model=model1)
ckpt.restore('/data/pretrained/cv/repnet/ckpt-88').expect_partial()
print('-'*20)
model1.pos_encoding = tf.tile(model1.pos_encoding, multiples=[1, 1, 512])
model1.pos_encoding2 = tf.tile(model1.pos_encoding2, multiples=[1, 1, 512])
print(model1.pos_encoding.shape)
print(model1.pos_encoding[0][0][:10]) # 0.02054866
test_inputs = np.random.randn(2, 64, 112, 112, 3).astype(np.float32)
model1(tf.convert_to_tensor(test_inputs))
print('end')
# predictions = model1.predict(test_inputs)
# model1.summary()

(1, 64, 1)
tf.Tensor([-0.00950385], shape=(1,), dtype=float32)
--------------------
(1, 64, 512)
tf.Tensor(
[0.02054866 0.02054866 0.02054866 0.02054866 0.02054866 0.02054866
 0.02054866 0.02054866 0.02054866 0.02054866], shape=(10,), dtype=float32)
1conv_layer: (2, 64, 7, 7, 1024)
2conv_layer: (2, 64, 7, 7, 512)
bn_layer: (2, 64, 7, 7, 512)
relu layer: (2, 64, 7, 7, 512)
end


In [26]:
os.system('rm -rf /data/bb')
model1.save("/data/bb", save_format='tf', include_optimizer=False)

1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, None, None, 512)
1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, None, None, 512)
1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, None, None, 512)
1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, None, None, 512)
1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, None, None, 512)
1conv_layer: (None, None, None, None, None)
2conv_layer: (None, None, None, None, 512)
bn_layer: (None, None, None, None, 512)
relu layer: (None, None, Non

INFO:tensorflow:Assets written to: /data/bb/assets


INFO:tensorflow:Assets written to: /data/bb/assets
/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model('/data/aa/')
# tflite_model = converter.convert()
# with open('/data/aa/repnet.tflite', 'wb') as f:
#     f.write(tflite_model)

In [43]:
!python3 -m tf2onnx.convert --opset 11 \
    --saved-model /data/bb --output /data/bb/test.onnx --verbose --debug

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-08-30 19:47:09,761 - WARNING - tf2onnx.tf_loader: '--tag' not specified for saved_model. Using --tag serve
2021-08-30 19:47:20,627 - INFO - tf2onnx.tf_loader: Signatures found in model: [serving_default].
2021-08-30 19:47:20,627 - WARNING - tf2onnx.tf_loader: '--signature_def' not specified, using first signature: serving_default
2021-08-30 19:47:20,629 - INFO - tf2onnx.tf_loader: Output names: ['output_1', 'output_2', 'output_3']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-08-30 19:47:25,525 - WARNING - tensorflow: From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Inst

2021-08-30 19:47:55,525 - VERBOSE - tf2onnx.optimizer.BackToBackOptimizer: BatchNormalization -20 (31->11), Const -60 (271->211), Squeeze -12 (17->5), Unsqueeze -12 (50->38)
2021-08-30 19:47:55,525 - VERBOSE - tf2onnx.optimizer: Apply einsum_optimizer
2021-08-30 19:47:55,658 - VERBOSE - tf2onnx.optimizer.EinsumOptimizer: no change
2021-08-30 19:47:55,658 - VERBOSE - tf2onnx.optimizer: Apply optimize_transpose
2021-08-30 19:47:55,809 - VERBOSE - tf2onnx.optimizer.TransposeOptimizer: no change
2021-08-30 19:47:55,809 - VERBOSE - tf2onnx.optimizer: Apply remove_redundant_upsample
2021-08-30 19:47:55,934 - VERBOSE - tf2onnx.optimizer.UpsampleOptimizer: no change
2021-08-30 19:47:55,934 - VERBOSE - tf2onnx.optimizer: Apply fold_constants
2021-08-30 19:47:56,072 - VERBOSE - tf2onnx.optimizer.ConstFoldOptimizer: Cast -2 (69->67), Const +2 (211->213)
2021-08-30 19:47:56,073 - VERBOSE - tf2onnx.optimizer: Apply const_dequantize_optimizer
2021-08-30 19:47:56,199 - VERBOSE - tf2onnx.optimizer.Con

In [47]:
import onnx
import onnx.helper as OH
import onnxruntime as rt
from PIL import Image
from collections import OrderedDict
import onnxruntime as rt

# sess = rt.InferenceSession('/data/bb/test.onnx', providers=["CPUExecutionProvider"])  
sess = rt.InferenceSession('/data/bb/test.onnx')  

In [31]:
import onnx

from onnx_tf.backend import prepare

In [ ]:
tf.convert_to_tensor(test_inputs)

In [34]:
test_inputs.shape

(2, 64, 112, 112, 3)

In [44]:
onnx_model = onnx.load('/data/bb/test.onnx')
tf_rep = prepare(onnx_model)

In [41]:
tf_rep.inputs, tf_rep.outputs

(['input_1'], ['output_1', 'output_2', 'output_3'])

In [42]:
tf_rep.graph

In [45]:
# tf_rep.export_graph("output_path")
tf_rep.run(test_inputs) 

ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/onnx_tf/backend_tf_module.py:99 __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    /usr/local/lib/python3.6/dist-packages/onnx_tf/backend.py:333 _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    /usr/local/lib/python3.6/dist-packages/onnx_tf/handlers/handler.py:59 handle  *
        return ver_handle(node, **kwargs)
    /usr/local/lib/python3.6/dist-packages/onnx_tf/handlers/backend/transpose.py:14 version_1  *
        return [cls.make_tensor_from_onnx_node(node, **kwargs)]
    /usr/local/lib/python3.6/dist-packages/onnx_tf/handlers/backend_handler.py:157 make_tensor_from_onnx_node  *
        return cls._run_tf_func(tf_func, inputs, attrs)
    /usr/local/lib/python3.6/dist-packages/onnx_tf/handlers/backend_handler.py:237 _run_tf_func  *
        return tf_func(**kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflo

In [15]:
model = onnx.load("/data/bb/test.onnx")
onnx.checker.check_model(model, full_check=True)
# onnx.helper.printable_graph(model.graph)

FileNotFoundError: [Errno 2] No such file or directory: '/data/aa/test.onnx'

In [16]:
sess.get_inputs()[0].name

'input_1'

In [30]:
dir(sess)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_inference_session',
 '_enable_fallback',
 '_fallback_providers',
 '_inputs_meta',
 '_model_bytes',
 '_model_meta',
 '_model_path',
 '_outputs_meta',
 '_overridable_initializers',
 '_profiling_start_time_ns',
 '_provider_options',
 '_providers',
 '_read_config_from_model',
 '_reset_session',
 '_sess',
 '_sess_options',
 '_sess_options_initial',
 'disable_fallback',
 'enable_fallback',
 'end_profiling',
 'get_inputs',
 'get_modelmeta',
 'get_outputs',
 'get_overridable_initializers',
 'get_profiling_start_time_ns',
 'get_provider_options',
 'get_providers',
 'get_session_options',
 'io_binding',
 'run',
 'run_with_

In [29]:
# test_inputs = tf.random.uniform((1, 64, 112, 112, 3))
test_inputs = np.random.randn(2, 64, 112, 112, 3).astype(np.float32)
inputs = {sess.get_inputs()[0].name: test_inputs}
outputs = [o.name for o in sess.get_outputs()]
predictions = sess.run(outputs, inputs)
predictions

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Transpose node. Name:'StatefulPartitionedCall/resnet_period_estimator_3/RE_temporal_bn_layers/FusedBatchNormV3__564' Status Message: perm: [ 0 3 1 2 ] does not align with rank of input data: 5

In [ ]:
%netron /data/aa/test.onnx 8228 600

In [ ]:
# inputs = tf.convert_to_tensor(test_inputs)

inputs = tf.random.uniform((1, 64, 112, 112, 3))
predictions = model1.predict(inputs)
model1.summary()
full_model = tf.function(lambda x: model1(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(inputs.shape, inputs.dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
# frozen_func.graph.as_graph_def()

In [ ]:
graph = frozen_func.graph.as_graph_def()

In [ ]:
operations = frozen_func.graph.get_operations()

In [ ]:
print("-" * 50)
print("Frozen model layers: ")
for op in operations:
    print(op.name)

In [ ]:
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="/data/nb_data/tmp/frozen_models",
    name="simple_frozen_graph1.pb",
    as_text=False)

In [ ]:
!python3 -m tf2onnx.convert --input /data/nb_data/tmp/frozen_models/simple_frozen_graph1.pb \
    --inputs x:0 --outputs Identity:0,Identity_1:0,Identity_2:0 \
    --opset 10 --output /data/nb_data/tmp/repnet3.onnx

In [ ]:
sess = rt.InferenceSession('/data/nb_data/tmp/repnet3.onnx')  

In [ ]:
%netron /data/nb_data/tmp/repnet3.onnx 8222

In [ ]:
%watermark -p onnxruntime

## References

- https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md
- https://segmentfault.com/a/1190000039936376
- https://github.com/onnx/tensorflow-onnx/issues/1606